In [ ]:
import base64
import requests
from PIL import Image
import PIL
import pytesseract
from pdf2image import convert_from_path
from wand.image import Image as wandImage
import os
import pandas as pd
from multiprocessing import Pool, cpu_count
import multiprocessing as mp
import numpy as np
from io import BytesIO
import cv2
from time import sleep, time
from functools import partial
import json

In [ ]:
models = ['gpt-4o-mini', 'gpt-4o', 'gpt-4-turbo']

api_keys = { 'gpt-4o-mini': "PLACEHOLDER", 'gpt-4o' : "PLACEHOLDER", 'gpt-4-turbo' : "PLACEHOLDER" }

# initialize a JSON based off of 
# {"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}

batch_list = []
ocr_df = pd.read_excel('ocr_df.xlsx', index_col=0)
for model in models:
    start_time = time()
    api_key = api_keys[model]

    headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"}
    
    for index, row in ocr_df.iterrows():
        directory = 'test_files'
        pdf_text = ''
        images = convert_from_path(f'{directory}/{row["item_filename"]}.pdf')
        for p_index, image in enumerate(images):
            # Convert the first image to a byte stream
            image_byte_array = BytesIO()
            image.save(image_byte_array, format='PNG')  # Save the image in PNG format
            image_byte_array = image_byte_array.getvalue()  # Get the byte value of the image
          
          # Encode the byte stream to base64
            base64_image = base64.b64encode(image_byte_array).decode('utf-8')
                      
            
            payload = {
              "model": f"{model}",
              "messages": [
                {
                  "role": "user",
                  "content": [
                    {
                      "type": "text",
                      "text": '''Please perform OCR on this image and return the text. Do not return any other text such as "Sure! Here’s the extracted text from the image:"'''
                    },
                    {
                      "type": "image_url",
                      "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                      }
                    }
                  ]
                }
              ],
              "max_tokens": 3000
            }
            
            # response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
            # pdf_text += response.json()['choices'][0]['message']['content']
            
            
            # {"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}
            batch_list.append({"custom_id": f"{model}_{row['item_filename']}_{p_index}", "method": "POST", "url": "/v1/chat/completions", "body": payload})   
            
            # print(pdf_text)
            # break
            
        # ocr_df.at[index, f'{model}'] = pdf_text
        # break
    
    print(model, time() - start_time)
print(batch_list)

In [ ]:
# save the ocr_df to a file
ocr_df.to_excel('ocr_df_gpt.xlsx')
ocr_df.to_excel('ocr_df.xlsx')